# Getting Started with SiReNetA and Overview

Authors: **Gorka Zamora-López** and **Matthieu Gilson**

---------------------

This notebook is part of an introductory tutorial for the use of *Stimulus-Response Network Analysis* ([SiReNetA](https://github.com/mb-BCA/SiReNetA)) to study the structure of complex networks:
1. **[Getting Started and Overview](1_GettingStarted.ipynb)**
2. *[Calculating Response to Stimulus and Metrics](2_Basics_StimRespMetrics.ipynb)*
3. *[Canonical Models](3_Basics_CanonMods.ipynb)*
4. *[Comparing Networks](4_UseCase_CompareNets.ipynb)*
5. *[Network Distance](5_UseCase_NetDist.ipynb)*
6. *[Weighted Networks](6_UseCase_WeighteNets.ipynb)* 

---------------------

### Outline
* Installation of the *SiReNetA* library.
* Familiarization with the library and getting information.exa
* Use case to analyze an example undirected network.

---------------------

## 1. Installing the *SiReNetA* library and dependencies

*SiReNetA* is built upon the array object definition of [NumPy](https://numpy.org) and it uses functions from the [SciPy](https://scipy.org) library. Running these tutorials also requires [Matplotlib](https://matplotlib.org) for plotting and visualization purposes. The installation of *SiReNetA* will check for the presence of these libraries and install them if not available, see the file 'requirements.txt' in [SiReNetA](https://github.com/mb-BCA/SiReNetA).

Some notebooks also require the library [GAlib](https://github.com/gorkazl/pyGAlib) for graph generation, manipulation and analysis in the context of classic graph theory.

To get started, the first thing we need is to load the libraries we will need to work. Start importing the built-in and third party libraries.

The following cell checks whether *SiReNetA* is already installed (in the same Python environment in which this notebook is running) and otherwise, the library will be installed using Python package installer [pip](https://pypi.org/project/pip/). 

>**Note**: the exclamation mark ( ! ) before `pip install galib` indicates the Jupyter notebook to run the line of code as a system command, same as if we would run it from a terminal window.

In [ ]:
# Check SiReNetA is installed in the current Python environment, otherwise install (NOTE that you can also create a dedicated python environment to install SiReNetA)

try:
    import sireneta
except:
    # Install from the GitHub repository
    print('Installing SiReNetA from GitHub ...')
    ! pip install git+https://github.com/mb-BCA/SiReNetA.git@master

# Make sure the library is properly installed by importing it
import sireneta as sna

## 2. Getting familiarised 

Information about the library, its modules and functions is found 'online' as usual. Type `help(module_name)` or `module_name?` in a cell of the notebook (or in an IPython interactive window) to access the corresponding information (docstrings).

Run the following cell to see the general overview of *SiReNetA*.

In [ ]:
# See the general description of sireneta library 
sna?

As seen, the library is organised into four user modules:

- responses.py
- metrics.py
- simulate.py
- tools.py

Run the following cell (uncommenting lines one-by-one) to see the description and a list of functions accessible in each module.

In [ ]:
## Explore the help functions of SiReNetA

#sna.responses?
#sna.metrics?
#sna.simulate?
#sna.tools?

Finally, check the description of individual functions, the expected parameters and their outputs. For example:

In [ ]:
#sna.responses.TransitionMatrix?
sna.responses.Resp_DiscreteCascade?
#sna.responses.Resp_LeakyCascade?
#sna.metrics.GlobalResponse?
#sna.metrics.TimeToPeak?
#sna.simulate.RandomWalk?


## 3. Example Analysis of a Network Stimulus-Response

The core idea behind *SiReNetA* is to reveal the properties of networks by probing how the nodes of a network respond localised perturbations. That is, to apply a stimulus of unit amplitude to one node and observe how the other nodes respond over time. The manner in which the stimulus propagates throughout the network depends on the dynamical propagation dynamical model selected. Network metrics are then extracted out of the pair-wise, node-wise or global responses, depending on whether nodes are stimulated individually or jointly.

For illustration, we will load the small graph (binary and undirected) depicted below and visualise the network response at various levels for the leaky (continuous) cascade propagation model. For this model, the time to peak provides an approximation for the geodesic distance [Zamora-López and Gilson](https://doi.org/10.1063/5.0202241).

> **Note**: Some comments are provided below to understand the figures in the following, but the reader is referred to the following notebooks for further details about how to use the functionalities of SiReNetA.


![Sample Graph, n=8 nodes](#)



In [ ]:
# Python standard / third-party library imports

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

We consider the network determined by the following binary matrix.

In [ ]:
# Load a small network (binary or unweighted)
net = np.loadtxt('../Data/Testnet_N8.txt', dtype=int)
N = len(net)

print(net)

In [ ]:
plt.figure()
plt.title('Connectivity matrix')
plt.imshow(net, cmap='gray_r')
plt.clim(0,net.max())
plt.colorbar()
plt.xlabel('node index')
plt.ylabel('node index')
plt.tight_layout()
plt.show()

Now choosing a propagation model

In [ ]:
# Check the spectral radius and the largest time constant for the leaky cascade propagation model
evs = np.linalg.eigvals(net)
evsnorms = np.zeros(N, float)
for i in range(N):
    evsnorms[i] = np.linalg.norm(evs[i])
evmax = evsnorms.max()
taumax = 1.0/evmax

print( 'Spectral radius:\t%2.5f' %evmax )
print( 'Largest possible tau:\t%2.5f' %taumax )

In [ ]:
# Set a leakage time-constant
tau = 0.8 * taumax

# Define the temporal resolution for the "simulation"
tfinal = 10
dt = 0.01
tpoints = np.arange(0.0, tfinal+dt, dt)
nsteps = len(tpoints)

# Calculate the pair-wise responses for a time span between 0 and tmax
resps_lc = sna.Resp_LeakyCascade(net, S0=1.0, tau=tau, tmax=tfinal, timestep=dt) 

In [ ]:
# Visualise the pair-wise response matrices at times t = 0.1, 0.3, 0.5, 1.0, 2.0, 3.0
maxresp = resps_lc.max()

tidxlist = [10,20,50,100,200,500]
plt.figure(figsize=[12,8])
for i, tidx in enumerate(tidxlist):
    t = tpoints[tidx]
    plt.subplot(2,3,i+1)
    plt.title('$\mathcal{R}(t)$ response matrix at t=%1.1f' %t)
    plt.imshow(resps_lc[tidx], cmap='gray_r')
    plt.clim(0,maxresp)
    plt.colorbar()
    plt.xticks(np.arange(N), np.arange(N)+1)
    plt.yticks(np.arange(N), np.arange(N)+1)
    plt.xlabel('source node')
    plt.ylabel('target node')
plt.tight_layout()

Above the matrices correspond to the response for pairs of source/target nodes at several times. The curves below represent the responses to a stimulation at node 1, for all nodes (including 1). Another representatoion is given below, where each row has been summed and teh resulting value plotted across time: this corresponds to the response felt by each node following the joint stimulation of all nodes.

In comparison, the black dashed curve represents the global network response rescaled by the number of weights. This shows how specific nodes differ from global response, being larger or smaller.

In [ ]:
# Compute and visualise the node responses
inresp, outresp = sna.NodeResponses(resps_lc, selfloops=True)
glbresp = sna.GlobalResponse(resps_lc)

plt.figure()
for i in range(N):
    plt.plot(tpoints, inresp[:,i], label='Node %d' %(i+1))
plt.plot(tpoints, glbresp / N, label='Global', c='k', ls='--', lw=2)
plt.xlabel('Time (a.u.)')
plt.ylabel('Input Node Response')
plt.legend(frameon=False)
plt.tight_layout()

We can try to explain these curves, for example by checking whether the degree of the node has an influence on the response cumulated over time, which corresponds to the area under the curve.

In [ ]:
# Compare the node response with the degree
deg = net.sum(axis=0)

noderesp = sna.AreaUnderCurve(inresp, timestep=dt)

plt.figure()
plt.plot(deg,noderesp, 'o')
plt.xlabel('Degree of Node')
plt.ylabel('Total Node Response (Area Under Curve)')
plt.tight_layout()

Now going into the detail of the response to the stimulation of a single node, we can plot the curve corresponding to each matrix element. Below is an exemple for all matrix elements corresponding to the column for source being node 1.

Again, we plot as a reference the output node response for node 1 that corresponds to the sum of the response felt by all nodes when stimulating node 1. Here it becomes clearer that some pathways from node 1 to some other nodes are faster or slower than the black dashed reference. This illustrates that the spatio-temporal properties of the response capture the network topological structure.

In [ ]:
# Visualise temporal responses of nodes i = 2, 3, ... , 8 to stimulus in j = 1

plt.figure()
for i in range(N):
    plt.plot(tpoints, resps_lc[:,0,i], label='Resp, (Node %d | 1)' %(i+1) )
plt.plot(tpoints, outresp[:,1] / N, label='Out Resp Node 1', c='k', ls='--', lw=2)
plt.xlabel('Time-step')
plt.ylabel('Conditional Response')
plt.legend(frameon=False)
plt.tight_layout()

Following, we can extract the time to peak for each pair of source/target nodes, which we compare to the geodesic distance calculated on the binary graph.

In [ ]:
# Compute the classical graph distance
import galib
dij = galib.FloydWarshall(net)

# Compute the time-to-peak distance
ttpdist = sna.Time2Peak(resps_lc, timestep=dt)

plt.figure(figsize=(10,2.4))
# Plot the adjacency matrix
plt.subplot(1,3,1)
plt.title('Adjacency matrix')
plt.imshow(net, cmap='gray_r')
plt.colorbar()

# Plot the classical graph distance
plt.subplot(1,3,2)
plt.title('Graph distance')
plt.imshow(dij)
plt.colorbar()

# Plot the time-2-peak distance matrix
plt.subplot(1,3,3)
plt.title('Time-to-peak distance')
plt.imshow(ttpdist)
plt.colorbar()

plt.tight_layout()